## Parte Computacional Punto 3
### Coordenadas parabólicas

Autores: Juan Fernando Riascos y Thomas Martinod

En este `.ipynb` se presenta brevemente la segunda parte del tercer numeral de la segunda tarea de Mecánica Clásica... Vamos a resolver numéricamente las ecuaciones de Lagrange para diferentes condiciones iniciales y analizar los resultados.

Inicialmente, vamos a usar `sympy` para definir las variables simbólicamente, luego aplicar `lambdiify` y así obtener funciones, de modo que luego se usará `scipy.integrate.odeint` para resolver las ecuaciones de Euler Lagrange (EL).

In [21]:
# Importe de librerías
import sympy as smp
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

from sympy.physics.mechanics import *
init_vprinting(pretty_print=True)

# Definimos las variables simbólicas
t, m = smp.symbols('t, m')

tau, sig = smp.symbols('tau, sigma', cls=smp.Function)
tau, sig = tau(t), sig(t)
tau

In [22]:
# Definimos las derivadas de las coordenadas
dtau = smp.diff(tau, t)
dsig = smp.diff(sig, t)

d2tau = smp.diff(dtau, t)
d2sig = smp.diff(dsig, t)

d2sig

In [23]:
# Definimos el Lagrangiano
L = smp.Rational(1,2) * m * (tau**2 + sig**2) * (dtau**2 + dsig**2)

Recordemos que las ecuaciones EL son:

$$    \frac{\partial L}{\partial u_i} - \frac{d}{dt}\left(\frac{\partial L}{\partial \dot{u}_i}\right) = 0 $$

o también se escriben:

$$L_{u_i} - \frac{d}{dt} L_{\dot{u}_i} = 0$$

Comencemos con $u_i = \tau$.

In [24]:
L_tau = smp.diff(L, tau)
L_tau

In [26]:
L_dtau = smp.diff(L, dtau)
L_dtau

In [34]:
dL_dtau = smp.diff(L_dtau, t)
dL_dtau

In [31]:
EL1 = smp.Eq(L_tau - dL_dtau, 0)
EL1

Ahora obtengamos la segunda ecuación de Euler Lagrange con $u_i = \sigma$:

In [32]:
L_sig = smp.diff(L, sig)

In [33]:
L_dsig = smp.diff(L, dsig)

In [35]:
dL_dsig = smp.diff(L_dsig, t)

In [36]:
EL2 = smp.Eq(L_sig - dL_dsig, 0)
EL2

Dadas estas dos ecuaciones diferenciales de segundo orde